## We will do the below task using word2vec custom model training
- Take one dataset from kaggle from any where(web)
- train you custom word2vec model
- then reduce dimesion of vectors into 2 dimesion using PCA(dimesion reduction)
- then plot the data using graph(plot 2-d data)
- check the similarity between the words

In [1]:
from gensim.models import Word2Vec, KeyedVectors
import gensim.downloader as api
import pandas as pd
import numpy as np
import os
import gensim
import nltk
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [2]:
model = api.load('word2vec-google-news-300')

In [3]:
# Loading ramayana from github
import os
import requests

def download_github_folder(repo_url, folder_path, local_dir):
    """
    Download a folder and its files from a GitHub repository.

    Args:
        repo_url (str): GitHub repository URL 
        folder_path (str): Path to the folder in the repository (e.g., 'path/to/folder').
        local_dir (str): Local directory to save the downloaded folder.
    Returns:
        None
    """
    # Extract owner and repo name from the URL
    url_parts = repo_url.rstrip('/').split('/')
    owner, repo = url_parts[-2], url_parts[-1]

    # GitHub API endpoint for the folder
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents/{folder_path}"

    # Send a GET request to the GitHub API
    response = requests.get(api_url)

    # Handle possible errors
    if response.status_code != 200:
        print(f"Failed to fetch folder contents. Error: {response.status_code}")
        return

    # Parse the response JSON
    contents = response.json()

    # Create the local directory if it doesn't exist
    os.makedirs(local_dir, exist_ok=True)

    for item in contents:
        if item['type'] == 'file':
            # Download the file
            file_url = item['download_url']
            file_path = os.path.join(local_dir, item['name'])
            print(f"Downloading {file_url} to {file_path}...")
            file_response = requests.get(file_url)
            with open(file_path, 'wb') as f:
                f.write(file_response.content)
        elif item['type'] == 'dir':
            # Recursively download subfolders
            subfolder_path = os.path.join(folder_path, item['name'])
            subfolder_local_dir = os.path.join(local_dir, item['name'])
            download_github_folder(repo_url, subfolder_path, subfolder_local_dir)

# Example Usage
repo_url = "https://github.com/cltk/sanskrit_parallel_gitasupersite"
folder_path = "ramayana"
local_dir = "/Users/praveensrivas/Documents/NLPwithGenAI/NLP_GEN_AI/Ramayan_book_data/"

download_github_folder(repo_url, folder_path, local_dir)


In [4]:
story = []
folder_path = r"/Users/praveensrivas/Documents/NLPwithGenAI/NLP_GEN_AI/Ramayan_book_data/"
for filename in os.listdir(folder_path):
  file_path = os.path.join(folder_path,filename)
  with open(file_path,encoding='unicode_escape') as f:
    corpus = f.read()
  raw_sent = sent_tokenize(corpus)
  for sent in raw_sent:
    story.append(simple_preprocess(sent))

In [5]:
custom_model = gensim.models.Word2Vec(window=3,min_count=7,vector_size=150,sg=1)

In [6]:
custom_model.build_vocab(story)

In [7]:
custom_model.corpus_count

18763

In [8]:
custom_model.train(story,total_examples=len(story), epochs=10)

(2609625, 5033470)

In [9]:
custom_model.wv["king"].shape

(150,)

In [10]:
# Reducing dimension of the data using TSNE
from sklearn.manifold import TSNE

num_dimension = 2

#vector and labels
vectors = np.asarray(custom_model.wv.vectors)
labels = np.asarray(custom_model.wv.index_to_key)

#apply tsne
tsne = TSNE(n_components=num_dimension, random_state=0)
vectors = tsne.fit_transform(vectors)


In [11]:

x_vals = [v[0] for v in vectors]
y_vals = [v[1] for v in vectors]

In [12]:
#Visualize embeddings with TSNE and Plotly
import plotly.graph_objs as go
fig = go.Figure()
trace = go.Scatter(x=x_vals,y=y_vals,mode='markers',text=labels)
fig.add_trace(trace=trace)
fig.update_layout(title="t-SNE of the Ramayan data word2vec embedding")
fig.show()


In [13]:
print(custom_model.wv.most_similar("rama"))

[('walking', 0.5740209221839905), ('elder', 0.5599085688591003), ('satrughna', 0.5542518496513367), ('prince', 0.5489961504936218), ('lakshamana', 0.5468811988830566), ('raghava', 0.5390234589576721), ('ah', 0.532893180847168), ('saumitri', 0.5212728977203369), ('raghavas', 0.5197228193283081), ('vibhishana', 0.5193439722061157)]


In [14]:
print(custom_model.wv.most_similar("sita"))

[('vaidehi', 0.7563008666038513), ('janaki', 0.6488282084465027), ('princess', 0.6323838233947754), ('lady', 0.5783380270004272), ('alas', 0.5761656761169434), ('mythili', 0.5652449131011963), ('janaka', 0.5519179701805115), ('crying', 0.5383111238479614), ('beloved', 0.5378361344337463), ('videha', 0.5372467637062073)]


In [15]:
print(custom_model.wv.doesnt_match(['sita', 'rama', 'laxman', 'hanuman']))

hanuman


In [16]:
# saving model
# custom_model.save("word2vec_custom_ramayan_model")
print(custom_model.wv.most_similar(positive=['rama'],topn=3))

[('walking', 0.5740209221839905), ('elder', 0.5599085688591003), ('satrughna', 0.5542518496513367)]


In [17]:
custom_model.wv.similarity('prince','rama')

0.54899615